Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 4.9 ms, sys: 461 µs, total: 5.36 ms
Wall time: 12.6 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/7666/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 198 ms, sys: 23.2 ms, total: 222 ms
Wall time: 2.12 s


[1.6979271673359064,
 1.3317870865436285,
 1.6677055819794762,
 1.6757303630371825,
 1.716576812433599,
 1.3723149428332597,
 1.2488599004634433,
 1.4360404794319326,
 1.5746572716091134,
 1.2570439414417733,
 1.9172551218314022,
 1.169041124854716,
 1.6613990521961628,
 1.2212167590639993,
 1.1054943719818917,
 1.4943000426362345,
 1.202364872717716,
 1.425727478799446,
 1.9458718380635198,
 1.4947994896362111,
 1.8871483762871764,
 1.8488089352465775,
 1.3613345896241589,
 1.6362056669447642,
 1.2799743574899707,
 1.3921313406598879,
 1.971954843810035,
 1.347781935331887,
 1.5699335925505715,
 1.1561278919460287,
 1.418183912032289,
 1.1806707343464917,
 1.8693734197954948,
 1.1714321317373226,
 1.790238325974097,
 1.5277921158627872,
 1.2520455980533365,
 1.660303351755522,
 1.774557089342736,
 1.9683038581687922,
 1.1864343155568813,
 1.0358760088675525,
 1.9038305538854297,
 1.9695836582232,
 1.1679605592484306,
 1.5705533783588836,
 1.0179146685930314,
 1.0851168151646062,
 1.74

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.5797389134960214,
 1.572313505710278,
 1.729529145970993,
 1.1662837009285307,
 1.0069174053728835,
 1.3853689528911177,
 1.0792440865008848,
 1.236956221467684,
 1.5081469267839513,
 1.6115374323490412,
 1.8748130698276069,
 1.56220808370121,
 1.5067362260521833,
 1.5736092231527508,
 1.3058811430042752,
 1.0202588974028806,
 1.6425359454961725,
 1.8497339132961284,
 1.7765280662136564,
 1.4639796297165577,
 1.7715138525019287,
 1.6539154170382844,
 1.5075682066456895,
 1.2822330182861537,
 1.8916673219871798,
 1.7844893284364152,
 1.5119763278435419,
 1.2159859076295796,
 1.7180795441122247,
 1.8864415966651409,
 1.7064570135339245,
 1.0452508243060663,
 1.2149705704377944,
 1.9373064004635092,
 1.9561484570698584,
 1.754518303603903,
 1.4175601334118149,
 1.8709179478225997,
 1.2999861178867829,
 1.5752207706053603,
 1.7078207848807803,
 1.7005876930025876,
 1.5872132305027749,
 1.91573898586278,
 1.8032767506873904,
 1.673244224160645,
 1.7753729653039483,
 1.3286220547883263,
 

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 232 ms, sys: 7.38 ms, total: 239 ms
Wall time: 2.86 s
